In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [89.9 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hi

In [2]:
# Install great expectations
!pip install great_expectations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 5.2 MB/s 
     |████████████████████████████████| 793 kB 47.5 MB/s 
     |████████████████████████████████| 139 kB 53.7 MB/s 
     |████████████████████████████████| 9.9 MB 35.1 MB/s 
     |████████████████████████████████| 109 kB 43.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 4.1 MB 32.8 MB/s 
     |████████████████████████████████| 381 kB 22.7 MB/s 
     |████████████████████████████████| 423 kB 41.3 MB/s 
     |████████████████████████████████| 127 kB 56.6 MB/s 
     |████████████████████████████████| 546 kB 53.5 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling

In [3]:
# Import SparkSession and great expectations 
from pyspark.sql import SparkSession
import great_expectations as ge

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# Start Spark session
from pyspark import SparkFiles

In [4]:
# Read the data into a Spark DataFrame.
customer_df = spark.read.csv('/content/customer_data.csv', sep=',', header=True)
customer_df.show()

+---+-----------+---------+--------------------+-------------+
| id| first_name|last_name|               email|          car|
+---+-----------+---------+--------------------+-------------+
|  0|       Jeff|  Gregory|johnsonmichael@ex...|        Buick|
|  1|      Robin|  Johnson|ernest54@example.net|        Lexus|
|  2|    Shannon| Thompson| jmathis@example.com|        Honda|
|  3|      Sandy|  Collier|thomaskaren@examp...|       Toyota|
|  4|     Morgan|  Simpson|allendakota@examp...|        Dodge|
|  5|  Katherine|  Rosario|tateamanda@exampl...|      Porsche|
|  6|      Riley| Castillo|brittneypeterson@...|        Honda|
|  7|     Justin|      Lee| lprince@example.org|          BMW|
|  8|    Suzanne|    Smith| bbarnes@example.com|         Jeep|
|  9|    Charles| Marshall|heididiaz@example...|      Porsche|
| 10|    Gregory|  Shelton|   tward@example.org|       Subaru|
| 11|Christopher|   Hudson|larrypeters@examp...|      Porsche|
| 12|        Kim|   Knight| xnelson@example.net|       

In [5]:
# Create the Great Expectations DataFrame
customer_df_ge = ge.dataset.SparkDFDataset(customer_df)
customer_df_ge.head()

,id,first_name,last_name,email,car
0,0,Jeff,Gregory,johnsonmichael@example.net,Buick
1,1,Robin,Johnson,ernest54@example.net,Lexus
2,2,Shannon,Thompson,jmathis@example.com,Honda
3,3,Sandy,Collier,thomaskaren@example.com,Toyota
4,4,Morgan,Simpson,allendakota@example.org,Dodge


In [6]:
# Test that each column doesn't contain null values.
for column in ['id', 'first_name', 'last_name', 'email', 'car']:
  print(customer_df_ge.expect_column_values_to_not_be_null(column=column)['success'])

True
True
True
False
False


In [7]:
# Use a function to remove the null values from a temporary view and saves them to a parquet file. 
def separate_nulls():
  # Create a temporary view
  customer_df.createOrReplaceTempView('customers')
  # Use Spark SQL to retrieve all the null values form the temporary view.
  transformed_df = spark.sql(
      """
      SELECT
        *
      FROM customers
      WHERE car IS NULL OR email IS NULL
      """
  )
  # Add the null values to a new temporary view that we'll reference later.
  transformed_df.createOrReplaceTempView('removed_customers')
  # Write the null values to a parquet file. 
  transformed_df.write.mode("overwrite").parquet("customers_null_car_or_email")
  print("Null values have been found and written to a temporary view, 'removed_customers', and a parquet file,'customers_null_car_or_email'.")

In [8]:
# Retrieve the "success" fields for the "email" and "car" columns. 
result = []
result.append(customer_df_ge.expect_column_values_to_not_be_null('email')['success'])
result.append(customer_df_ge.expect_column_values_to_not_be_null('car')['success'])
# If the "success" results are False then call the separate_null() function. 
# If there are no null values print "success".
if False in result:
  separate_nulls()
else:
  print("success")

Null values have been found and written to a temporary view, 'removed_customers', and a parquet file,'customers_null_car_or_email'.


In [9]:
# Use a function to transform the customers temporary view if the id not in the removed_customers.
def transform_data():
  transformed_df = spark.sql(
      """
      SELECT
        UPPER(first_name) AS FIRST_NAME,
        UPPER(last_name) AS LAST_NAME,
        UPPER(email) AS EMAIL,
        UPPER(car) AS CAR,
        UNIX_TIMESTAMP() AS CREATED_AT,
        UNIX_TIMESTAMP() AS UPDATED_AT,
        'FROM CLIENT' AS SOURCE
      FROM customers
      WHERE id not in (select id from removed_customers)
      """
  )
  return transformed_df

In [10]:
# Call the transformation function and convert the transformed DataFrame into a Great Expectations DataFrame 
transformed_df = transform_data()
transformed_df_ge = ge.dataset.SparkDFDataset(transformed_df)
# Retrieve the "success" fields for the all the modified columns in the transformed DataFrame.
transformed_result = []
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('EMAIL')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('CAR')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('CREATED_AT')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('UPDATED_AT')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('source')['success'])
# If the "success" field is 'False' print "Failed", if not then write the transformed DataFrame to a parquet file.
if False in transformed_result:
  print('Failed')
else:
  transformed_df.write.mode("overwrite").parquet("customers")
  print("Success: Wrote the transformed data to a parquet file, 'customers'.")


Success: Wrote the transformed data to a parquet file, 'customers'.


In [11]:
# Read in our customers parquet data
customers_p_df=spark.read.parquet('customers')
customers_p_df.show(truncate=False)

+-----------+---------+----------------------------+-------------+----------+----------+-----------+
|FIRST_NAME |LAST_NAME|EMAIL                       |CAR          |CREATED_AT|UPDATED_AT|SOURCE     |
+-----------+---------+----------------------------+-------------+----------+----------+-----------+
|JEFF       |GREGORY  |JOHNSONMICHAEL@EXAMPLE.NET  |BUICK        |1660247133|1660247133|FROM CLIENT|
|ROBIN      |JOHNSON  |ERNEST54@EXAMPLE.NET        |LEXUS        |1660247133|1660247133|FROM CLIENT|
|SHANNON    |THOMPSON |JMATHIS@EXAMPLE.COM         |HONDA        |1660247133|1660247133|FROM CLIENT|
|SANDY      |COLLIER  |THOMASKAREN@EXAMPLE.COM     |TOYOTA       |1660247133|1660247133|FROM CLIENT|
|MORGAN     |SIMPSON  |ALLENDAKOTA@EXAMPLE.ORG     |DODGE        |1660247133|1660247133|FROM CLIENT|
|KATHERINE  |ROSARIO  |TATEAMANDA@EXAMPLE.NET      |PORSCHE      |1660247133|1660247133|FROM CLIENT|
|RILEY      |CASTILLO |BRITTNEYPETERSON@EXAMPLE.COM|HONDA        |1660247133|1660247133|FRO